In [1]:
"""
%reload_ext autoreload
%autoreload 2

import sys
sys.path.append('../')

import pandas as pd
from utils.plotting import *

import warnings
warnings.filterwarnings('ignore')

repo_path = '/Users/marinasiebold/Library/Mobile Documents/com~apple~CloudDocs/Studium/Bird_Research/Rbeast/Python/py_src'
if repo_path not in sys.path:
    sys.path.insert(0, repo_path)

print(sys.path)
import Rbeast
"""

"\n%reload_ext autoreload\n%autoreload 2\n\nimport sys\nsys.path.append('../')\n\nimport pandas as pd\nfrom utils.plotting import *\n\nimport warnings\nwarnings.filterwarnings('ignore')\n\nrepo_path = '/Users/marinasiebold/Library/Mobile Documents/com~apple~CloudDocs/Studium/Bird_Research/Rbeast/Python/py_src'\nif repo_path not in sys.path:\n    sys.path.insert(0, repo_path)\n\nprint(sys.path)\nimport Rbeast\n"

In [2]:
%reload_ext autoreload
%autoreload 2

import sys
sys.path.append('../')

import matplotlib.pyplot as plt
import Rbeast as rb
import pandas as pd
import geopandas as gpd
from datetime import datetime, timedelta

import warnings
warnings.filterwarnings('ignore')


In [3]:
path = '../../../01_Data/datasets/time_series_27_species_week_counts.csv'
#path = '../../../01_Data/datasets/time_series_feldsperling_week_counts.csv'
df = pd.read_csv(path)

In [90]:
from math import isnan
def decimal_year_to_datetime(decimal_year):
    if isnan(decimal_year):
        return None
    year = int(decimal_year)
    remainder = decimal_year - year
    start_of_year = datetime(year, 1, 1)
    start_of_next_year = datetime(year + 1, 1, 1)
    year_duration = start_of_next_year - start_of_year
    date_time = start_of_year + timedelta(seconds=remainder * year_duration.total_seconds())
    return date_time.strftime('%d-%m-%Y')


In [ ]:
"""
o = rb.beast(bird.sighting_ratio, deltat='7 days', period='1 year',
             start='2018-01-01', end='2022-12-31',
             hasOutlier=True, print_options=False, print_progress=False)
fig = plt.figure(figsize=(15, 10))
fig = rb.plot(o, fig=fig, title=f'BEAST analysis for {species} in {grid_id}')
"""

### **Changepoint map of a Bergente**

#### Calculate change points

In [5]:
species = 'Bergente'
bird = df[df['name_species'] == species]
all_grids = bird.eea_grid_id.unique()

In [6]:
slices = {}
for grid in all_grids:
    slice = bird[bird['eea_grid_id'] == grid].sighting_ratio
    slices[species, grid] = slice

In [91]:
results = []
for (species, grid_id), sighting_ratios in slices.items():
    o = rb.beast(sighting_ratios, deltat='7 days', period='1 year', 
                 start='2018-01-01', end='2022-12-31',
                 hasOutlier=False,
                 print_progress=False, quiet=True)
    num_cp = int(o.trend.ncp_median)
    cps = o.trend.cp[:num_cp]
    cp_probs = o.trend.cpPr[:num_cp]
    
    pos_cps = 0
    neg_cps = 0
    date_pos = []
    prob_pos = []
    date_neg = []
    prob_neg = []
    for cp, cp_prob in zip(cps, cp_probs):
        closest_index = np.argmin(np.abs(o.time -  cp))
        pos_slope_prob = o.trend.slpSgnPosPr[closest_index]
        neg_slope_prob = 1- pos_slope_prob - o.trend.slpSgnZeroPr[closest_index]
        if pos_slope_prob > neg_slope_prob:
            pos_cps += 1
            date_pos = decimal_year_to_datetime(cp)
            prob_pos.append(round(cp_prob, 3))
        else:
            neg_cps += 1
            date_neg = decimal_year_to_datetime(cp)
            prob_neg.append(round(cp_prob, 3))
    result = {'species': species, 'grid_id': grid_id,
              'pos_cp': pos_cps, 'dates_pos': date_pos, 'probs_pos': prob_pos,
              'neg_cp': neg_cps, 'dates_neg': date_neg, 'probs_neg': prob_neg
              }
    results.append(result)

beast_results = pd.DataFrame(results)
beast_results

,species,grid_id,pos_cp,dates_pos,probs_pos,neg_cp,dates_neg,probs_neg
0,Bergente,50kmE4000N2500,0,[],[],1,10-02-2020,[0.114]
1,Bergente,50kmE4000N2550,0,[],[],1,24-12-2018,[0.211]
2,Bergente,50kmE4000N2600,0,[],[],2,12-10-2021,"[0.929, 0.591]"
3,Bergente,50kmE4000N2850,0,[],[],1,30-03-2020,[0.116]
4,Bergente,50kmE4000N2900,0,[],[],1,20-01-2020,[0.118]
...,...,...,...,...,...,...,...,...
225,Bergente,50kmE4600N3400,1,09-02-2021,[0.181],0,[],[]
226,Bergente,50kmE4600N3450,0,[],[],1,20-01-2020,[0.116]
227,Bergente,50kmE4650N3050,1,26-01-2021,[0.405],0,[],[]
228,Bergente,50kmE4650N3100,1,18-01-2022,[0.702],0,[],[]


#### Load grid info

In [92]:
eea_shapefile_path = '../../../01_Data/shp_files/grids/eea_europe_grids_50km/inspire_compatible_grid_50km.shp'
eea_grid = gpd.read_file(eea_shapefile_path)
eea_grid = eea_grid.to_crs('EPSG:4326')

germany_switzerland_bbox = eea_grid.cx[5.210942:15.669926, 45.614516:55.379499]

eea_grid_filtered = eea_grid[eea_grid.intersects(germany_switzerland_bbox.unary_union)].copy()
eea_grid_filtered.rename(columns={'cellcode': 'grid_id'}, inplace=True)
eea_grid_filtered.reset_index(drop=True, inplace=True)

In [95]:
merged_df = pd.merge(beast_results, eea_grid_filtered, on='grid_id')
merged_df.drop(columns=['eoforigin', 'noforigin', 'gid'], inplace=True)

gdf = gpd.GeoDataFrame(merged_df, geometry='geometry', crs='EPSG:4326')
gdf = gdf.reset_index(drop=True)
gdf

,species,grid_id,pos_cp,dates_pos,probs_pos,neg_cp,dates_neg,probs_neg,geometry
0,Bergente,50kmE4000N2500,0,[],[],1,10-02-2020,[0.114],"POLYGON ((5.89422 45.53284, 5.86021 45.98253, ..."
1,Bergente,50kmE4000N2550,0,[],[],1,24-12-2018,[0.211],"POLYGON ((5.86021 45.98253, 5.82545 46.43207, ..."
2,Bergente,50kmE4000N2600,0,[],[],2,12-10-2021,"[0.929, 0.591]","POLYGON ((5.82545 46.43207, 5.78994 46.88148, ..."
3,Bergente,50kmE4000N2850,0,[],[],1,30-03-2020,[0.116],"POLYGON ((5.63974 48.67790, 5.60003 49.12673, ..."
4,Bergente,50kmE4000N2900,0,[],[],1,20-01-2020,[0.118],"POLYGON ((5.60003 49.12673, 5.55939 49.57547, ..."
...,...,...,...,...,...,...,...,...,...
225,Bergente,50kmE4600N3400,1,09-02-2021,[0.181],0,[],[],"POLYGON ((14.21996 53.63443, 14.26508 54.08284..."
226,Bergente,50kmE4600N3450,0,[],[],1,20-01-2020,[0.116],"POLYGON ((14.26508 54.08284, 14.31139 54.53123..."
227,Bergente,50kmE4650N3050,1,26-01-2021,[0.405],0,[],[],"POLYGON ((14.63728 50.46819, 14.68196 50.91663..."
228,Bergente,50kmE4650N3100,1,18-01-2022,[0.702],0,[],[],"POLYGON ((14.68196 50.91663, 14.72772 51.36499..."


In [101]:
import geopandas as gpd
import plotly.express as px

fig = px.choropleth_mapbox(
    gdf, 
    geojson=gdf, 
    locations=gdf.index, 
    color="pos_cp",
    color_continuous_scale='YlGn',
    center={"lat": gdf.geometry.centroid.y.mean(), "lon": gdf.geometry.centroid.x.mean()},
    zoom=10,
    opacity=0.8,
    hover_data={'dates_pos': True, 'probs_pos': False, 'pos_cp': True},
)

fig.update_layout(
    mapbox=dict(
        style='open-street-map',
        zoom=4
    ), 
    showlegend=True,
    margin=dict(l=0, r=0, t=0, b=0)
)

fig.update_traces(marker_line_width=0)
fig.show()

In [103]:
import geopandas as gpd
import plotly.express as px

fig = px.choropleth_mapbox(
    gdf, 
    geojson=gdf, 
    locations=gdf.index, 
    color="neg_cp",
    color_continuous_scale='reds',
    center={"lat": gdf.geometry.centroid.y.mean(), "lon": gdf.geometry.centroid.x.mean()},
    zoom=10,
    opacity=0.8,
    hover_data={'dates_neg': True, 'probs_neg': False, 'neg_cp': True},
)

fig.update_layout(
    mapbox=dict(
        style='open-street-map',
        zoom=4
    ), 
    showlegend=True,
    margin=dict(l=0, r=0, t=0, b=0)
)

fig.update_traces(marker_line_width=0)
fig.show()